In [1]:
import warnings
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
# dataframe은 cus, cprl, dcd,  weight로 구성
# dcd = 1이면 cus <- cprl 방향, dcd = 2이면 cus -> cprl 방향
data = pd.read_csv('network_data.csv')

In [ ]:
red = [] # 빨간 색으로 표시하고 싶은 노드의 이름 넣기
blue = ['CASH'] # 파란 색으로 표시하고 싶은 노드의 이름 넣기
# 기본 색은 하늘색

In [2]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data['minmax'] = scaler.fit_transform(data[['weight']]) +0.0001

In [ ]:
def make_network_data(df):
  d02_net = df[['cus','cprl','dcd','minmax']]
  d02_net.rename(colums = {'cus':'to', 'cprl':'from'},inplace = True)

  #방향에 따라 노드 순서 재배열 하기
  mask = d02_net['dcd'] == 2 #조건 정의
  d02_net.loc[mask,['from','to']] = d02_net.loc[mask, ['to','from']].values
  d02_net.drop(['dcd'],inplace=True, axis =1)

  #엣지 만들기
  d02_net['weighted_edge'] = list(zip(d02_net['from'], d02_net['to'], d02_net['minmax']))
  d02_net['edge'] = list(zip(d02_net['from'],d02_net['to']))

  return d02_net

In [ ]:
#연결된 노드를 확인하기 위해 네트워크 구축
g = nx.from_pandas_edgelist(make_network_data(data), 'from', 'to', edge_attr ='minmax')

#conneted_component 찾기 : 각 리스트 원소는 connected graph 안에 존재한다.
sorted(nx.connected_components(g))

In [ ]:
#connected graph 하나씩 그리기 - weighted edges version
connected_nodes = sorted(nx.connected_components(g))
for i in range(len(connected_nodes)):
  nodes_list = connected_nodes[i]

  graph1 = data[data['cus'].isin(nodes_list)|data['cprl'].isin(nodes_list)]
  case_net = make_network_data(graph1)

  G1 = nx.DiGraph()
  G1.add_nodes_from(nodes_list)
  G1.add_weighted_edges_from(list(case_net['weighted_edge']))

  plt.figure(figsize=(12,12))
  edge_widths = [d['weight']*10 for u,v,d in G1.edges(data=True)]

  node_colors = []
  for node in G1.nodes:
    if node not in red:
      node_colors.append('lightblue')
    elif node in blue:
      node_colors.append('dodgerblue')
    else:
      node_colors.append('#fc8d62')

  layout = nx.fruchterman_reingold_layout(G1, iterations =20) #nx.spring_layout(G1) 혹은 nx.kamada_kawai_layout(G1) 등 사용 가능
  nx.draw_networkx_nodes(G1,layout, node_size =100, node_color = node_colors)
  nx.draw_networkx_edges(G1, layout, arrowsize = 30, width=edge_widths, edge_color = 'gray')
  nx.draw_networkx_labels(G1, layout) # 생략 가능 : node의 이름 띄우기
  plt.axis('off')

  graph_title = 'CASE_ID:' + str(i)
  plt.title(graph_title)

  plt.show()

In [ ]:
#connected graph 하나씩 그리기 - not weighted edges version
connected_nodes = sorted(nx.connected_components(g))
for i in range(len(connected_nodes)):
  nodes_list = connected_nodes[i]

  graph1 = data[data['cus'].isin(nodes_list)|data['cprl'].isin(nodes_list)]
  case_net = make_network_data(graph1)

  G1 = nx.DiGraph()
  G1.add_nodes_from(nodes_list)
  G1.add_edges_from(list(case_net['edge']))

  plt.figure(figsize=(12,12))

  node_colors = []
  for node in G1.nodes:
    if node not in red:
      node_colors.append('lightblue')
    elif node in blue:
      node_colors.append('dodgerblue')
    else:
      node_colors.append('#fc8d62')

  layout = nx.fruchterman_reingold_layout(G1, iterations =20) #nx.spring_layout(G1) 혹은 nx.kamada_kawai_layout(G1) 등 사용 가능
  nx.draw_networkx_nodes(G1,layout, node_size =100, node_color = node_colors)
  nx.draw_networkx_edges(G1, layout, arrowsize = 30, edge_color = 'gray')
  nx.draw_networkx_labels(G1, layout) # 생략 가능 : node의 이름 띄우기
  plt.axis('off')

  graph_title = 'CASE_ID:' + str(i)
  plt.title(graph_title)

  plt.show()